Select NDSI_Snow_Cover from MOD10A.006 for Red Mountain Pass, CO  (37°53′56″N 107°42′43″W) for 1 Oct 2000 - 1 Oct 2015

In [2]:
import re
import os
import urllib
import netCDF4
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from ipywidgets import interact
from urllib.error import HTTPError
from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta

TIMEFRAME = [datetime(2000, 10, 1), datetime(2015, 10, 1)]
MODIS_BASE_URL = "http://n5eil01u.ecs.nsidc.org:80/opendap/MOST/MOD10A1.006/"
MODIS_AREA_IDENTIFIER = "h09v05"

def daterange(timeframe):
    for n in range(int ((timeframe[1] - timeframe[0]).days)):
        yield timeframe[0] + timedelta(n)  

def form_smashed_date(date):
    return str(date.year) + str(date.month).zfill(2) + str(date.day).zfill(2)
          
def form_dotted_date(date):
    return str(date.year) + "." + str(date.month).zfill(2) + "." + str(date.day).zfill(2)

def dataset_lats(dataset):
    return dataset["Latitude"][:,:]

def dataset_lons(dataset):
    return dataset["Longitude"][:,:]

def dataset_values(dataset):
    return dataset["NDSI_Snow_Cover"][:,:]


In [ ]:
#%matplotlib inline
#def draw_figure(dataset):
#    plt.figure(figsize=(15,15))
#    b = Basemap(projection='stere', lat_0=50,lon_0=-105,resolution='c', llcrnrlon=-110, llcrnrlat=35,urcrnrlon=-105, urcrnrlat=40)
#    b.drawstates()
#    b.drawcounties()
#    b.pcolor(dataset_lats(dataset), dataset_lons(dataset), dataset_values(dataset), latlon=True)
    
#@interact(num=widgets.IntSlider(min=0,max=2318,value=0,step=1,continuous_update=False))
#def show_it(num=0):
#    dataset = netCDF4.Dataset(get_filesystem_dataset(num))
#    draw_figure(dataset)

#dataset = netCDF4.Dataset(get_filesystem_dataset(5))
#draw_figure(dataset)

In [ ]:
def download_dataset(url, filename, missing):
    try:
        file, headers = urllib.request.urlretrieve(url, "./MODIS/" + filename)
        print("downloaded " + filename)
    except HTTPError as e:
        print("Failed to download " + filename + ", error: " + str(e))
        write_to_missing(missing, url)

def download_xml(url, filename):
    file, headers = urllib.request.urlretrieve(url, "./XML/" + filename)
    print("downloaded " + url + " (xml)")
    return file

def write_to_missing(missing, filename):
    print("Adding " + filename + " to missing files")
    missing.write(filename + "\n")
        
def process_modis():
    missing_files = open("./MODIS/missing_files.txt","a")
    for date in daterange(TIMEFRAME):
        dotted_date = form_dotted_date(date)
        print("Downloading xml for " + dotted_date)
        xml_url = MODIS_BASE_URL + dotted_date + "/catalog.xml"
        filename = form_smashed_date(date) + "_catalog.xml"
        try:
            xml = download_xml(xml_url, filename)
        except HTTPError as e:
            print("Failed to download " + xml_url +", " + str(e))
            print("Couldn't get xml for " + dotted_date + ", skipping")
            write_to_missing(missing_files, xml_url)
            continue
        xml = open("./XML/" + filename)
        tree = ET.parse(xml)
        root = tree.getroot()
        
        for child in root[2]:
            match = re.search(r"h09v05.+\.hdf$", child.attrib["name"])
            if match:
                dap_filename = child.attrib["name"]
                filename = "MODIS_h09v05_" + form_smashed_date(date) + ".hdf"
                #make sure that we haven't already downloaded this file
                if not os.path.isfile("./MODIS/" + filename):
                    print("Found " + dap_filename + ", downloading")
                    download_dataset(MODIS_BASE_URL + dotted_date + "/" + dap_filename, filename, missing_files)
                else:
                    print("Already downloaded " + dap_filename + ", skipping")

process_modis()

downloaded http://n5eil01u.ecs.nsidc.org:80/opendap/MOST/MOD10A1.006/2000.10.01/catalog.xml (xml)
Already downloaded MOD10A1.A2000275.h09v05.006.2016069002515.hdf, skipping
downloaded http://n5eil01u.ecs.nsidc.org:80/opendap/MOST/MOD10A1.006/2000.10.02/catalog.xml (xml)
Already downloaded MOD10A1.A2000276.h09v05.006.2016069022312.hdf, skipping
downloaded http://n5eil01u.ecs.nsidc.org:80/opendap/MOST/MOD10A1.006/2000.10.03/catalog.xml (xml)
Already downloaded MOD10A1.A2000277.h09v05.006.2016069032442.hdf, skipping
downloaded http://n5eil01u.ecs.nsidc.org:80/opendap/MOST/MOD10A1.006/2000.10.04/catalog.xml (xml)
downloaded http://n5eil01u.ecs.nsidc.org:80/opendap/MOST/MOD10A1.006/2000.10.05/catalog.xml (xml)
Already downloaded MOD10A1.A2000279.h09v05.006.2016069060252.hdf, skipping
downloaded http://n5eil01u.ecs.nsidc.org:80/opendap/MOST/MOD10A1.006/2000.10.06/catalog.xml (xml)
Already downloaded MOD10A1.A2000280.h09v05.006.2016069065825.hdf, skipping
downloaded http://n5eil01u.ecs.nsidc.